In [7]:
# imports
import pandas as pd
import glob
import os
import json

In [15]:
# Path to games_by_year folder
folder_path = './data/games_by_year/'

# get all files
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Read and combine the CSV's
df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

# Combine all into DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Show head
combined_df.head()

id                   slug                     name    released  \
0   58172             spacewar-2                Spacewar!  1962-01-01   
1  379437        zeusophthekroos           ZeusOfTheCrows  1970-01-01   
2   56681  the-oregon-trail-1971  The Oregon Trail (1971)  1971-12-03   
3   55012                 empire                   Empire  1971-01-01   
4   58174         computer-space           Computer Space  1971-12-01   

   metacritic  rating  ratings_count  playtime  added  twitch_count  \
0         NaN    3.78             22         1     39             0   
1         NaN    0.00              4         0     16             0   
2         NaN    3.62             34         0     87             0   
3         NaN    3.43              6         0     31             0   
4         NaN    2.91              8         0     29             0   

   youtube_count                                        description website  \
0              0  Spacewar! is a pioneering space combat video g...     NaN   
1              0  This should not be searcheable. If you find th...     NaN   
2              0  The Oregon Trail is a computer game originally...     NaN   
3              0  Empire (or Classic Empire) is a turn-based war...     NaN   
4              0  Computer Space is a space combat arcade game d...     NaN   

  reddit_url                                          platforms  \
0        NaN                                                NaN   
1        NaN                                                NaN   
2        NaN  Commodore / Amiga, Classic Macintosh, Apple II...   
3        NaN                    Commodore / Amiga, Atari ST, PC   
4        NaN                                                NaN   

                  genres                                            tags  \
0                 Arcade                     2D, Retro, Space, 2-players   
1                  Indie                                             NaN   
2  Adventure, Simulation                                    Singleplayer   
3               Strategy                       Singleplayer, Multiplayer   
4                 Arcade  Space, combat, battle, history, rocket, colors   

                                         developers  \
0                                     Steve Russell   
1                                               NaN   
2  MECC, Minnesota Educational Computing Consortium   
3       Walter Bright, Mark Baldwin, Peter Langston   
4                                Syzygy Engineering   

                                          publishers  year  
0                                      Steve Russell  1962  
1                                                NaN  1970  
2         Gameloft, Brøderbund, The Learning Company  1971  
3  New World Computing, Killer Bee Software, Inte...  1971  
4                                 Nutting Associates  1971

In [16]:
# Path to company top games
json_folder = './data/company top games/'

# get all files
json_files = glob.glob(os.path.join(json_folder, '*.json'))

# List to hold all game entries
company_games_list = []

for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
        # Only process files with actual game results
        if 'results' in data and isinstance(data['results'], list):
            # Extract company name from filename
            filename = os.path.basename(file)
            company_name = filename.replace('_top_games.json', '').replace('_', ' ')
            
            for game in data['results']:
                game['company'] = company_name
                company_games_list.append(game)

# Convert to DataFrame
company_games_df = pd.json_normalize(company_games_list)

# Get the first 5 unique companies
first_5_companies = company_games_df['company'].unique()[:5]

# Filter the DataFrame to only show rows from these companies
top_5_company_games = company_games_df[company_games_df['company'].isin(first_5_companies)]

# Show a preview of their games
top_5_company_games[['company', 'name', 'released', 'rating']].sort_values(by='company').head(20)

company                                name    released  rating
0   343 Industries   Halo: The Master Chief Collection  2014-11-11    4.25
1   343 Industries                 Halo 2: Anniversary  2014-11-11    4.21
2   343 Industries    Halo: Combat Evolved Anniversary  2011-11-15    3.97
3   343 Industries                              Halo 4  2012-11-06    3.82
4   343 Industries                   Halo 5: Guardians  2015-10-27    3.74
5   343 Industries       Halo Wars: Definitive Edition  2017-04-20    3.72
6   343 Industries                       Halo Infinite  2021-12-08    3.65
7   343 Industries                Halo: Spartan Strike  2015-04-16    3.28
8   343 Industries               Halo: Spartan Assault  2013-07-18    3.12
9   343 Industries                       Halo Waypoint  2010-09-09    2.72
19      Activision                          River Raid  1982-01-01    4.08
18      Activision  MechWarrior 2: 31st Century Combat  1995-01-01    4.12
17      Activision     Crash Bandicoot N. Sane Trilogy  2017-06-30    4.13
16      Activision             Spyro Reignited Trilogy  2018-11-13    4.13
15      Activision                 Destiny 2: Forsaken  2018-09-04    4.14
12      Activision                 X-Men 2: Clone Wars  1995-01-01    4.17
13      Activision           Call of Duty: Strike Team  2013-09-05    4.17
11      Activision         Civilization: Call to Power  1999-03-01    4.23
10      Activision                              Enduro  1983-02-01    4.33
14      Activision                      Interstate '76  1997-02-28    4.15

In [19]:
# Path to top 50 games per genre
genre_folder = './data/top 50 games per genre/'
genre_files = glob.glob(os.path.join(genre_folder, '*.json'))

# List to hold all game data
genre_games_list = []

for file in genre_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)

        # Extract genre from the first genre tag in each game
        for game in data:
            if 'genres' in game and isinstance(game['genres'], list) and len(game['genres']) > 0:
                main_genre = game['genres'][0].get('name', 'Unknown')
            else:
                main_genre = 'Unknown'

            game['main_genre'] = main_genre
            genre_games_list.append(game)

# Convert to DataFrame
genre_games_df = pd.json_normalize(genre_games_list)

# Preview
genre_games_df[['main_genre', 'name', 'released', 'rating']].head()

main_genre                      name    released  rating
0     Action        Grand Theft Auto V  2013-09-17    4.47
1     Action  The Witcher 3: Wild Hunt  2015-05-18    4.65
2     Action        Tomb Raider (2013)  2013-03-05    4.06
3     Action                    Portal  2007-10-09    4.50
4    Shooter             Left 4 Dead 2  2009-11-17    4.09

In [22]:
# Path to the file (adjust if needed)
file_path = './data/top 5000 critic reviewed games/top_5000_critic_reviewed_games.json'

# Load the JSON data
with open(file_path, 'r', encoding='utf-8') as f:
    critic_data = json.load(f)

# Convert to DataFrame
critic_df = pd.json_normalize(critic_data)

# Preview
critic_df[['name', 'released', 'metacritic', 'rating', 'genres']].head()

name    released  metacritic  rating  \
0  The Legend of Zelda: Ocarina of Time  1998-11-21          99    4.39   
1                    Soulcalibur (1998)  1998-07-30          98    0.00   
2                           Soulcalibur  1998-07-30          98    4.37   
3                     Baldur's Gate III  2023-08-03          97    4.47   
4                         Metroid Prime  2002-11-17          97    4.37   

                                              genres  
0  [{'id': 4, 'name': 'Action', 'slug': 'action',...  
1  [{'id': 6, 'name': 'Fighting', 'slug': 'fighti...  
2  [{'id': 4, 'name': 'Action', 'slug': 'action',...  
3  [{'id': 3, 'name': 'Adventure', 'slug': 'adven...  
4  [{'id': 4, 'name': 'Action', 'slug': 'action',...